In [4]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from prefect_gcp import GcpCredentials

In [ ]:
@task(retries=3)
def extract_from_gcs():
    """Download trip data from GCS"""
    gcs_path = "youtube_data"
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.get_directory(from_path=gcs_path, local_path=f"../data/")
    return Path(f"../data/{gcs_path}")

@task()
def write_bq(df: pd.DataFrame,color: str) -> None:
    """Write DataFrame to BiqQuery"""

    gcp_credentials_block = GcpCredentials.load("zoom-gcp-creds")

    df.to_gbq(
        destination_table= f"dezoomcamp.rides_{color}",
        project_id="adept-storm-375515",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        chunksize=500_000,
        if_exists="append",
    )

@flow()
def etl_gcs_to_bq():
    """Main ETL flow to load data into Big Query"""
            path = extract_from_gcs(x, year, month)
            df = transform(path)
            write_bq(df,x)

if __name__ == "__main__":
    etl_gcs_to_bq()

In [2]:
def extract_from_gcs():
    """Download trip data from GCS"""
    gcs_path = "youtube_data"
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.get_directory(from_path=gcs_path, local_path=f"../data/")
    return Path(f"../data/{gcs_path}")

In [5]:
path = extract_from_gcs()

AttributeError: 'coroutine' object has no attribute 'get_directory'

In [ ]:
def write_bq(df: pd.DataFrame):
    """Write DataFrame to BiqQuery"""

    gcp_credentials_block = GcpCredentials.load("zoom-gcp-creds")

    df.to_gbq(
        destination_table= f"dtc-de-youtube.youtube_data_all",
        project_id="dtc-de-youtube",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        chunksize=500_000,
        if_exists="append",
    )

In [7]:
print(Path('youtube_data').glob('*.parquet'))

<generator object Path.glob at 0x7f16d04d25f0>


In [10]:
import re
s = "file path .parquet"
print(re.findall(r'(?:/[^/]+)+?/\w+\.\w+', s))

[]


In [ ]:
def extract_from_gcs():
    """Download trip data from GCS"""
    gcs_path = "youtube_data"
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.get_directory(from_path=gcs_path, local_path=f"../data/")
    return Path(f"../data/{gcs_path}")

extract_from_gcs()

In [12]:
!pip install pandas-gbq==0.18.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 50.6 MB/s eta 0:00:0000:0100:01


In [13]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

/home/iamuser/anaconda3/lib/python3.9/site-packages/google/api_core/gapic_v1/config.py:23: RuntimeWarning: coroutine 'Block.load' was never awaited
  import grpc
/home/iamuser/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/iamuser/anaconda3/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.w

In [ ]:
# TODO(developer): Set table_id to the ID of the table to create.
table_id = "dtc-de-youtube.youtube_data_all.youtube_reviews

# Set the encryption key to use for the destination.
# TODO: Replace this key with a key you have created in KMS.
kms_key_name = "projects/{}/locations/{}/keyRings/{}/cryptoKeys/{}".format(
#     "cloud-samples-tests", "us", "test", "test"
# )
job_config = bigquery.LoadJobConfig(
    autodetect=True, source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
)
uri = "gs://dtc_data_lake_youtube_dtc-de-youtube/youtube_data/20230314.parquet"
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.
load_job.result()  # Waits for the job to complete.
destination_table = client.get_table(table_id)
print("Loaded {} rows.".format(destination_table.num_rows))